In [27]:
pip install dnspython

   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/307.7 kB ? eta -:--:--
   --- ----------------------------------- 30.7/307.7 kB 435.7 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/307.7 kB 871.5 kB/s eta 0:00:01
   ---------------------------------- ----- 266.2/307.7 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 307.7/307.7 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [35]:
import subprocess
import time
import socket
import random
import dns.resolver
import re

def resolve_domain(domain):
    """
    Resolves a domain name to an IP address.
    """
    try:
        # Remove protocols (http:// or https://) from the domain if present
        domain = re.sub(r'^https?://', '', domain)
        return socket.gethostbyname(domain)
    except socket.gaierror as e:
        print(f"Error resolving domain: {e}")
        return None

def reverse_dns_lookup(ip_address):
    """
    Performs a reverse DNS lookup to find the domain name associated with an IP address.
    """
    try:
        return socket.gethostbyaddr(ip_address)[0]
    except socket.herror:
        return "No PTR record found"

def is_valid_ip(ip_address):
    """
    Checks if the given string is a valid IPv4 or IPv6 address.
    """
    try:
        socket.inet_pton(socket.AF_INET, ip_address)  # IPv4
        return True
    except socket.error:
        try:
            socket.inet_pton(socket.AF_INET6, ip_address)  # IPv6
            return True
        except socket.error:
            return False

def is_valid_domain(domain):
    """
    Checks if the given string is a valid domain name.
    """
    domain = re.sub(r'^https?://', '', domain)  # Remove protocols if present
    try:
        socket.gethostbyname(domain)
        return True
    except socket.gaierror:
        return False

def ping_server(server_address, port=None):
    """
    Pings the server if no port is provided, or checks if the specific port is open.
    """
    try:
        if port is None:
            # Ping the server using the ping command
            output = subprocess.run(
                ["ping", "-c", "1", server_address],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            return output.returncode == 0
        else:
            # Check if the port is open using a TCP socket
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
                sock.settimeout(3)  # 3 seconds timeout
                result = sock.connect_ex((server_address, port))
                return result == 0
    except Exception as e:
        print(f"Error checking server: {e}")
        return False

def restore_connection(server_address, port=None):
    """
    Placeholder function for restoring a connection. Replace with actual commands as needed.
    """
    if port:
        service = get_service_name(port)
        print(f"Attempting to restore connection to {server_address} on port {port} ({service})...")
    else:
        print(f"Attempting to restore connection to {server_address}...")
    # Example: Reconnect to a VPN, restart a service, etc.
    # subprocess.run(["your_command", "arg1", "arg2"])

def monitor_connection(server_address, ports, check_interval=10, stealth_mode=True):
    """
    Monitors the connection to the server on the specified ports and attempts to restore 
    the connection if any ports are down. In stealth mode, scans are randomized.
    """
    reverse_dns = reverse_dns_lookup(server_address)
    print(f"Monitoring {server_address} (Domain: {reverse_dns})")
    
    while True:
        all_ports_up = True
        if stealth_mode:
            random.shuffle(ports)  # Randomize the order of ports to avoid detection
        for port in ports:
            if not ping_server(server_address, port):
                service = get_service_name(port)
                print(f"{time.ctime()}: Connection to {server_address} (Domain: {reverse_dns}) on port {port} ({service}) is down.")
                restore_connection(server_address, port)
                all_ports_up = False
            else:
                service = get_service_name(port)
                print(f"{time.ctime()}: Connection to {server_address} (Domain: {reverse_dns}) on port {port} ({service}) is up.")
            
            if stealth_mode:
                time.sleep(random.uniform(0.5, 3.0))  # Random delay between port checks
        
        if all_ports_up:
            print(f"{time.ctime()}: All monitored ports on {server_address} (Domain: {reverse_dns}) are up.")
        time.sleep(check_interval)

def get_service_name(port):
    """
    Returns the service name for a given port number.
    """
    try:
        return socket.getservbyport(port)
    except OSError:
        return "Unknown service"

def parse_ports(port_input):
    """
    Parses a string of ports and ranges (e.g., '443, 80, 3306-3308, 455 - 600, 3212')
    into a list of ports, handling irregular spacing and ranges.
    """
    ports = set()  # Use a set to avoid duplicates
    parts = port_input.split(',')
    
    for part in parts:
        part = part.strip().replace(' ', '')  # Remove any spaces around the dash
        if '-' in part:
            # It's a range
            start, end = part.split('-')
            ports.update(range(int(start), int(end) + 1))
        else:
            # It's a single port
            ports.add(int(part))
    
    return sorted(ports)

def dns_map(domain):
    """
    Retrieves a full DNS map for a given domain, including A, AAAA, MX, NS, CNAME, and TXT records.
    """
    print(f"\nDNS Map for {domain}:")

    record_types = ['A', 'AAAA', 'MX', 'NS', 'CNAME', 'TXT']

    for record_type in record_types:
        try:
            answers = dns.resolver.resolve(domain, record_type)
            for rdata in answers:
                print(f"{record_type} Record: {rdata}")
        except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
            print(f"{record_type} Record: None found")

def get_user_choice(prompt, options):
    """
    Repeatedly prompts the user until a valid choice is entered.
    """
    while True:
        choice = input(prompt).strip()
        if choice in options:
            return choice
        print("Invalid input. Please try again.")

if __name__ == "__main__":
    # Ask the user if they want to use an IP address or domain name
    choice = get_user_choice(
        "Do you want to use (1) an IP address or (2) a domain name? Enter 1 or 2: ",
        {"1", "2"}
    )
    
    if choice == "1":
        server_address = input("Enter the server IP address: ").strip()
        if not is_valid_ip(server_address):
            print("Invalid IP address format. Exiting.")
            exit(1)
        reverse_dns = reverse_dns_lookup(server_address)
        print(f"Resolved IP address {server_address} to domain: {reverse_dns}")
    elif choice == "2":
        domain = input("Enter the domain name: ").strip()
        if not is_valid_domain(domain):
            print("Invalid domain name. Exiting.")
            exit(1)
        server_address = resolve_domain(domain)
        if server_address is None:
            print("Unable to resolve the domain. Exiting.")
            exit(1)
        reverse_dns = reverse_dns_lookup(server_address)
        print(f"Resolved domain {domain} to IP address {server_address}")
        print(f"Resolved IP address {server_address} to domain: {reverse_dns}")
        
        # Display the full DNS map for the domain
        dns_map(domain)
    
    # Ask the user for port input or choose all ports
    port_input_choice = get_user_choice(
        "Enter (1) to monitor all ports or (2) to enter specific ports or ranges: ",
        {"1", "2"}
    )
    
    if port_input_choice == "1":
        ports = list(range(1, 65536))  # All ports from 1 to 65535
    elif port_input_choice == "2":
        port_input = input("Enter the ports or ranges (e.g., '443, 80, 3306-3308, 455 - 600, 3212'): ").strip()
        ports = parse_ports(port_input)
    
    # Ask the user if they want to enable stealth mode
    stealth_mode = input("Enable stealth mode (y/n)? ").strip().lower() == 'y'
    
    # Set the interval between checks
    check_interval = 10  # Default: Check every 10 seconds
    monitor_connection(server_address, ports, check_interval, stealth_mode)


Do you want to use (1) an IP address or (2) a domain name? Enter 1 or 2:  2
Enter the domain name:  itpt.co.uk


Resolved domain itpt.co.uk to IP address 77.68.74.155
Resolved IP address 77.68.74.155 to domain: server.itpt.co.uk

DNS Map for itpt.co.uk:
A Record: 77.68.74.155
AAAA Record: None found
MX Record: 1 itpt-co-uk.mail.protection.outlook.com.
NS Record: ns-uk.1and1-dns.biz.
NS Record: ns-uk.1and1-dns.com.
NS Record: ns-uk.1and1-dns.co.uk.
NS Record: ns-uk.1and1-dns.org.
CNAME Record: None found
TXT Record: "v=spf1 include:servers.mcsv.net ?all"
TXT Record: "v=spf1 include:spf.protection.outlook.com -all"
TXT Record: "google-site-verification=_keIvVX2qfBqM4Hz5qXRKXoF0z7zm_tcKpjHXJ8KDz4"


Enter (1) to monitor all ports or (2) to enter specific ports or ranges:  2
Enter the ports or ranges (e.g., '443, 80, 3306-3308, 455 - 600, 3212'):  443, 80, 8080, 3306
Enable stealth mode (y/n)?  y


Monitoring 77.68.74.155 (Domain: server.itpt.co.uk)
Tue Aug 20 20:22:59 2024: Connection to 77.68.74.155 (Domain: server.itpt.co.uk) on port 8080 (Unknown service) is down.
Attempting to restore connection to 77.68.74.155 on port 8080 (Unknown service)...
Tue Aug 20 20:23:05 2024: Connection to 77.68.74.155 (Domain: server.itpt.co.uk) on port 3306 (Unknown service) is down.
Attempting to restore connection to 77.68.74.155 on port 3306 (Unknown service)...
Tue Aug 20 20:23:08 2024: Connection to 77.68.74.155 (Domain: server.itpt.co.uk) on port 80 (http) is up.
Tue Aug 20 20:23:10 2024: Connection to 77.68.74.155 (Domain: server.itpt.co.uk) on port 443 (https) is up.
Tue Aug 20 20:23:22 2024: Connection to 77.68.74.155 (Domain: server.itpt.co.uk) on port 443 (https) is up.
Tue Aug 20 20:23:23 2024: Connection to 77.68.74.155 (Domain: server.itpt.co.uk) on port 80 (http) is up.
Tue Aug 20 20:23:27 2024: Connection to 77.68.74.155 (Domain: server.itpt.co.uk) on port 8080 (Unknown service) 

KeyboardInterrupt: 